# Annoy Analysis:
Annoy (Approximate Nearest Neighbors Oh Yeah) is an efficient technique for finding nearest neighbors in large datasets, particularly in applications involving vectors, such as machine learning and information retrieval.

## How Annoy Works:

Data (typically vectors) are added to an index. Each vector represents a numerical representation of an item.

Tree Division: Annoy builds multiple search trees, where each tree is constructed based on a random sample of the data. The idea is to partition the space into regions, allowing for faster neighbor searches.

Search Algorithm:
When querying a vector, Annoy searches through the constructed trees. It traverses the trees to find the nearest neighbors.
The algorithm is designed to be fast, using approximation, which means it might not always find the exact nearest neighbors but frequently finds very close neighbors efficiently.

In [1]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=553319 sha256=728aea7959a44b96b0ce334a4e91dddd7be7213506c9cf4fb47b0eccd63a90db
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [2]:
# Import Libs
import pandas as pd
from annoy import AnnoyIndex

In [3]:
df = pd.read_excel('rice_w2vec_3mer_dataset.xlsx')

In [4]:
df.head(2)

,circName,circID,gene,isoform,stress,tissue,chr,start,end,strand,...,wc_3mer_55,wc_3mer_56,wc_3mer_57,wc_3mer_58,wc_3mer_59,wc_3mer_60,wc_3mer_61,wc_3mer_62,wc_3mer_63,wc_3mer_64
0,osa-circ1-OS01T0723400,1:30167620-30167771_+,OS01T0723400,OS01T0723400-01,-,multipleTissue,1,30167620,30167771,+,...,-21.976927,-12.450649,-8.376489,-9.521074,28.565338,7.25165,-18.073032,-6.670159,-3.779034,1.302005
1,osa-circ2-OS03T0223400,3:6461672-6462146_-,OS03T0223400,OS03T0223400-01,-,multipleTissue,3,6461672,6462146,-,...,24.437080,-68.490463,20.158427,14.177653,81.437811,-4.52417,21.867150,-10.532807,-39.172970,63.339612


In [5]:
dimension = 64

# Cold stress dataset

In [19]:
df_cold = df.query('stress == "-" or stress == "cold"').replace('-', 'control')

In [20]:
df_vec = df_cold.drop(['seq','tissue','chr','start','end','strand','start_anno','circID', 'gene', 'isoform', 'width', 'detection_score', 'stress_detection_score','end_anno', 'antisense', 'algorithm', 'exonSeq',	'predAA',	'miRNA',	'superCircRNARegion'], axis=1)
df_vec

,circName,stress,wc_3mer_1,wc_3mer_2,wc_3mer_3,wc_3mer_4,wc_3mer_5,wc_3mer_6,wc_3mer_7,wc_3mer_8,...,wc_3mer_55,wc_3mer_56,wc_3mer_57,wc_3mer_58,wc_3mer_59,wc_3mer_60,wc_3mer_61,wc_3mer_62,wc_3mer_63,wc_3mer_64
0,osa-circ1-OS01T0723400,control,-8.660600,-17.171770,-20.124799,10.607480,-0.194991,1.568289,3.896590,7.622823,...,-21.976927,-12.450649,-8.376489,-9.521074,28.565338,7.251650,-18.073032,-6.670159,-3.779034,1.302005
1,osa-circ2-OS03T0223400,control,-4.623322,-42.058749,-73.491472,-51.222795,83.974883,-47.219114,-20.222901,29.187759,...,24.437080,-68.490463,20.158427,14.177653,81.437811,-4.524170,21.867150,-10.532807,-39.172970,63.339612
2,osa-circ3-OS11T0210300,control,-23.932238,-17.885510,-54.202845,15.682650,-30.424396,13.430986,-15.981544,27.324571,...,-34.355928,13.556080,-50.537661,-30.186757,64.202528,54.450606,-24.835176,2.558572,-9.287752,-25.051924
3,osa-circ4-OS02T0200900,control,-42.541243,-44.121847,-54.312436,-37.621338,53.242821,6.651359,-9.533580,22.141600,...,74.941825,-37.716935,-19.644171,36.562092,62.745595,14.111886,46.643984,-12.691884,40.033801,-0.980058
4,osa-circ5-OS05T0494800,control,-9.970732,20.265143,-4.832509,39.979788,-9.345131,-2.708796,13.024784,8.689754,...,-49.435175,33.124836,-4.937962,-21.755058,-0.672400,39.981767,-57.044162,-18.266155,-12.560292,-41.720146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63043,osa-circ63044--,control,-36.129064,-13.508832,0.005943,-11.045796,34.350577,-12.069487,11.820539,24.216416,...,71.354353,-16.542512,2.488324,37.899663,7.770607,-1.294477,27.140534,-35.586348,42.826113,1.914022
63044,osa-circ63045--,control,-81.022365,16.294278,78.293930,78.380937,31.556227,34.236833,50.813806,30.031006,...,117.117149,79.811355,-51.199810,132.507834,32.852214,35.095596,34.001745,-10.350830,162.961049,-95.637550
63045,osa-circ63046--,control,-46.754698,10.391865,38.252494,51.685485,21.559602,32.210430,50.047768,5.315855,...,61.597804,66.569489,-50.207208,85.853744,44.182827,53.164509,15.902078,18.139559,116.782359,-77.869255
63046,osa-circ63047--,control,-19.736442,-30.662747,-21.114486,7.117133,24.859926,13.406382,-1.219863,-7.344230,...,22.584763,-23.609748,-10.552717,18.842086,36.384463,-20.618450,16.068993,1.198586,2.631869,0.869086


In [21]:
df_vec['stress'].value_counts()

,count
stress,
control,50187
cold,5724


In [32]:
cold_sequences = df_vec[df_vec['stress'] == 'cold']
control_sequences = df_vec[df_vec['stress'] == 'control']

# Controlled sample of "control" sequences
control_sample = control_sequences.sample(n=5724, random_state=40)

# Combine "cold" and "control" sequences
combined_sequences = pd.concat([cold_sequences, control_sample])

# Initialize the Annoy index
t = AnnoyIndex(dimension, 'angular')
for i in range(len(combined_sequences)):
    t.add_item(i, combined_sequences.iloc[i, 2:2 + dimension].values)
t.build(10)

# Initialize data structures
similarities = {}
n_neighbors = 5  # Desired number of neighbors

valid_neighbors_count = []  # Count of valid neighbors
cold_without_neighbors = 0  # Counter for sequences with no valid neighbors
total_neighbors_found = 0  # Counter for found neighbors
sequences_with_less_than_5_neighbors = []  # For sequences with less than 5 valid neighbors

cold_count = 0  # Count of neighbors in the "cold" group
control_count = 0  # Count of neighbors in the "control" group

# Process the "cold" sequences
for stress_index in cold_sequences.index:
    adjusted_index = combined_sequences.index.get_loc(stress_index)

    # Search for neighbors
    neighbors = t.get_nns_by_item(adjusted_index, n_neighbors + 1)
    valid_neighbors = []

    for i in neighbors:
        if combined_sequences.iloc[i]['circName'] != combined_sequences.iloc[adjusted_index]['circName']:
            valid_neighbors.append(i)
        if len(valid_neighbors) == n_neighbors:
            break

    # Check if 5 valid neighbors were found
    if len(valid_neighbors) < n_neighbors:
        valid_neighbors_count.append(0)
        cold_without_neighbors += 1
        sequences_with_less_than_5_neighbors.append(combined_sequences.iloc[adjusted_index]['circName'])
    else:
        valid_neighbors_count.append(len(valid_neighbors))

    total_neighbors_found += len(valid_neighbors)  # Total neighbors found
    similarities[stress_index] = valid_neighbors  # Add the found neighbors for the sequence

    # Count neighbors for "cold" and "control"
    for i in valid_neighbors:
        neighbor_condition = combined_sequences.iloc[i]['stress']
        if neighbor_condition == 'cold':
            cold_count += 1
        elif neighbor_condition == 'control':
            control_count += 1

# Display the count of valid neighbors per "cold" sequence
print("Valid neighbors count per 'cold' sequence:")
print(valid_neighbors_count)

# Display the number of sequences with no neighbors
print(f'Number of iterated cold stress sequences without neighbors: {cold_without_neighbors}')
print(f'Total number of neighbors found: {total_neighbors_found}')  # Verifying the total number of neighbors found

# Print the sequences with less than 5 valid neighbors
print("Sequences with less than 5 valid neighbors:")
print(sequences_with_less_than_5_neighbors)

# Creating the final DataFrame
data = []

# Populate the DataFrame with found neighbors
for stress_index, neighbors in similarities.items():
    cold_sequence = combined_sequences.iloc[combined_sequences.index.get_loc(stress_index)]['circName']

    # If the "cold" sequence found less than 5 valid neighbors
    if len(neighbors) == 0:
        data.append({
            'seq_ref_cold': cold_sequence,
            'nearest_neighbor_circrna': '',
            'condition': ''
        })
    else:
        # Add the neighbors to the DataFrame
        for i in range(len(neighbors)):
            neighbor_sequence = combined_sequences.iloc[neighbors[i]]['circName']
            neighbor_condition = combined_sequences.iloc[neighbors[i]]['stress']
            data.append({
                'seq_ref_cold': cold_sequence,
                'nearest_neighbor_circrna': neighbor_sequence,
                'condition': neighbor_condition
            })

# Add sequences with no valid neighbors to the DataFrame
for seq in sequences_with_less_than_5_neighbors:
    data.append({
        'seq_ref_cold': seq,
        'nearest_neighbor_circrna': '',
        'condition': ''
    })

# Create the final DataFrame
df_results = pd.DataFrame(data)

# Similarity analysis
total_neighbors = len(similarities) * n_neighbors
num_cold_sequences = len(cold_sequences)

print(f'Number of iterated cold stress sequences: {num_cold_sequences}')
print(f'Total number of neighbors in the "cold" group: {cold_count}')
print(f'Total number of neighbors in the "control" group: {control_count}')
print(f'Percentage of neighbors in the "cold" group: {cold_count / total_neighbors * 100:.2f}%')
print(f'Percentage of neighbors in the "control" group: {control_count / total_neighbors * 100:.2f}%')

# Verifying the number of rows in the final DataFrame
print(f'Total number of rows in df_results: {len(df_results)}')

df_results

Valid neighbors count per 'cold' sequence:
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

,seq_ref_cold,nearest_neighbor_circrna,condition
0,osa-circ491-OS01T0588900,osa-circ24329-OS03T0188100,control
1,osa-circ491-OS01T0588900,osa-circ3250-OS01T0607200,control
2,osa-circ491-OS01T0588900,osa-circ30582-OS08T0453700,control
3,osa-circ491-OS01T0588900,osa-circ54283-OS01T0845100,control
4,osa-circ491-OS01T0588900,osa-circ21516-OS06T0646700,control
...,...,...,...
28615,osa-circ62840-OS08T0408200,osa-circ3141-OS07T0671800,control
28616,osa-circ62840-OS08T0408200,osa-circ62799-OS07T0661400,control
28617,osa-circ62840-OS08T0408200,osa-circ61641-OS04T0623300,control
28618,osa-circ62840-OS08T0408200,osa-circ43551-OS05T0102900,cold


In [33]:
df_results.to_excel('neighbors_results_cold_rice.xlsx', index=False)